# wdc: a WCPS Query Generator library for Python

This library allows users to generate WCPS queries purely on Python; and users can send these queries to rasdaman servers to get the corresponding data.

This notebook aims to teach the basic usage of the library.

## Creating Queries

First of all, we will start with our imports.
From the wdc package, we import Query and AxisSubset classes. Query is the class we use to generate WCPS queries, meanwhile we will utilize AxisSubset class to subset our data. 

In [1]:
# Import necessary classes
from wdc import Query, AxisSubset

### First Query

Our first example is a very simple query, where we ask for all data of the coverage.

Firstly, we will create a Query object. We do so by providing it a list of strings, which should consist of names of coverages that we want to query.

We can then print the query to see its WCPS equivalent. Alternatively, user can utilize the get_wcps() method, which does the same.

In [2]:
# Creating a query

# First step is to create a query object, by providing list of coverages to query.
# For this example we will utilize cloud_image coverage

q = Query(["cloud_image"])

# Printing the query object converts it to its WCPS equivalent

print(q)

# Alternatively, you can use the get_wcps() method to get the WCPS query as a string
# print(q.get_wcps())

for $c in ( cloud_image )
return ( $c  )


It is also possible to go through multiple coverages in one single query. This can be done by simply adding more names to the list of coverages we provide for the Query object.

Currently, it is not possible to have multiple iterator variables (i.e the $c in the WCPS query above) go through multiple coverages at the same time. Only one variable will go through all coverages that is provided.

In [3]:
# We provide more than one coverage name.

q = Query(["cloud_image", "AvgLandTemp"])

print(q)

for $c in (cloud_image,AvgLandTemp)
return ( $c  )


### Subsetting Data

We can subset the data by using the setSubset() method. We need to pass an array of AxisSubset objects, one for each axis of the datacube.

We will talk more about how to get the axes of a datacube later on this tutorial, namely in the Connection object. For now, we provide you the information that "cloud_image" consists of the following axes:

"ansi": [2022-08-09T08:50:00.000Z, 2022-10-17T02:05:00.000Z] \
"Lat": [29.98, 60] \
"Long": [-10, 20.02]

Following the format of:

"axis": [lowerBound, upperBound]

We can provide an AxisSubset object by passing it first the name of the axis, a starting point, and a stopping point. If no stopping point is provided, AxisSubset will act only as a point that points to wherever the starting point is specified. 

In [4]:
# Focusing on a single coverage for now.
q = Query(["cloud_image"])

# Creating subsets for each axis
ansi_subset = AxisSubset("ansi", "2022-09-01") # This subset only consist of one point, the date 2022-09-01
lat_subset = AxisSubset("Lat", 45, 60) # This subset is a range from 45 to 60
long_subset = AxisSubset("Long", -5, 20) # A range from -5 to 20

# Set the subsets to our query from above
q.set_subset([ansi_subset, lat_subset, long_subset])

# Print query, this will have the subset applied to the return value
print(q)

for $c in ( cloud_image )
let $subset := [ansi("2022-09-01"), Lat(45:60), Long(-5:20)]
return ( $c  [ $subset ] )


Of course, user can decide which axes they want to subset, it does not have to be all of them. It is also possible to do the subset in a single line, at the cost of readability.

In [5]:
q.set_subset([AxisSubset("ansi", "2022-09-01"), AxisSubset("Lat", 45, 60), AxisSubset("Long", -5, 20)])

# Printing the query, this will give us the same query as above.
print(q)

for $c in ( cloud_image )
let $subset := [ansi("2022-09-01"), Lat(45:60), Long(-5:20)]
return ( $c  [ $subset ] )


### Encoding the return value

We can utilize WCPS's encode() function to encode the return value in different ways. For example, we can ask rasdaman server to return the data to us in the form of .PNG image. For this purposes, Query class has a subclass Types. It includes some of the supported output types of rasdaman, such as TIFF, PNG and JPEG.

In [6]:
# Some examples from the Query.Types subclass
print(Query.Types.jpeg) # Prints "image/jpeg"


# Continuing from the previous query
# We set a return type for our query
q.encode(Query.Types.png)
print(q)

image/jpeg
for $c in ( cloud_image )
let $subset := [ansi("2022-09-01"), Lat(45:60), Long(-5:20)]
return encode ( ( $c  [ $subset ] ) , "image/png" )


Lastly, we can manually define what our query will return. We do so by utilizing the set_return_value() method.

Passing a value or a string will make the query return whatever is passed.

In [7]:
# Creating a new query.
life_query = Query(["AvgLandTemp"])

life_query.set_return_value(42)

print(life_query)

for $c in ( AvgLandTemp )
return ( ( 42 )  )


### Compressing Datacube to a Single Value: Aggregation Methods

"wdc" implements WCPS aggregation methods such as avg(), max() and min() in Python.

To apply an aggregation method, we can use the set_aggregation_method() function. The aggregation methods supported by "wdc" is provided in Query.AggregationMethod class.

Note that setting an aggregation method will make the query ignore any encoding that might have been set before. We can reset the aggregation method by calling reset_aggregation_method().

In [8]:
# Create a query
agg_query = Query(["AvgLandTemp"])
agg_query.set_subset([AxisSubset("ansi", "2014-07"),
                     AxisSubset("Lat", -20, 30),
                     AxisSubset("Long", 10, 30)])

# Set an aggregation method
agg_query.set_aggregation_method(Query.AggregationMethod.max)
print(agg_query)
print() # Empty line

# Query ignores the encoding because of the aggregation method
agg_query.encode(Query.Types.png)
print(agg_query)
print() # Empty line

# Resetting aggregation methods brings encoding back.
agg_query.reset_aggregation_method()
print(agg_query)
print() # Empty line

for $c in ( AvgLandTemp )
let $subset := [ansi("2014-07"), Lat(-20:30), Long(10:30)]
return max ( ( $c  [ $subset ] ) )

for $c in ( AvgLandTemp )
let $subset := [ansi("2014-07"), Lat(-20:30), Long(10:30)]
return max ( ( $c  [ $subset ] ) )

for $c in ( AvgLandTemp )
let $subset := [ansi("2014-07"), Lat(-20:30), Long(10:30)]
return encode ( ( $c  [ $subset ] ) , "image/png" )



### Constructing Coverages

We provide the Query.CoverageConstructor class to implement WCPS' powerful feature of coverage constructors. This allows users to create new coverages on the fly.

A coverage constructor object has a "name"; a list of axises that it will contain, that users can specify with attribute "over"; and the values the coverage will have along its axes, i.e "values". We can get its WCPS equivalent by calling get_wcps() or converting it to a string.

This class contains no setters as all attributes are publicly available and can be changed.

When setting the return values, users have to use minimal WCPS code. The axes provide an iterator each, which is named "$p{name of axis}", and we can use these iterators to define values.

In [9]:
# A coverage constructor.

constructor = Query.CoverageConstructor("new_coverage", [AxisSubset("x", 0, 100), AxisSubset("y", 0, 100)], "$px + $py")

print(constructor.get_wcps())

# Create a query
constructor_query = Query(["AvgLandTemp"])

# Set the return value to the constructor we defined
constructor_query.set_return_value(constructor)

# Encode it to png, when sent to rasdaman servers, this will return a gradient image.
constructor_query.encode(Query.Types.png)

print()
print(constructor_query)

coverage new_coverage
over $px x(0:100), $py y(0:100)
values $px + $py

for $c in ( AvgLandTemp )
return encode ( ( ( coverage new_coverage
over $px x(0:100), $py y(0:100)
values $px + $py )  ) , "image/png" )


### Complex Return Statements: switch-case

The "wdc" library has a way of implementing WCPS switch statements in Python. Using the add_switch_case() method we can include different cases that return something different than the others.

The function requires a boolean expression, and then a coverage expression (basically what the case will return). We can also make the case default by setting the "default" flag to True.

In [10]:
# Create query
complexQuery = Query(["AvgLandTemp"])

# Create cases, all cases return structs with RGB values.
complexQuery.add_switch_case("$c = 99999", "{red: 255, green: 255, blue: 255}")
complexQuery.add_switch_case("18 > $c", "{red: 0, green: 0, blue: 255}")
complexQuery.add_switch_case("23 > $c", "{red: 255, green: 255, blue: 0}")
complexQuery.add_switch_case("30 > $c", "{red: 255, green: 140, blue: 0}")   

# Default case, it will ignore the boolean expression
complexQuery.add_switch_case("", "{red: 255, green: 0, blue: 0}", default=True)

complexQuery.encode(Query.Types.png)

print(complexQuery)


for $c in ( AvgLandTemp )
return encode ( ( 
switch
case $c = 99999 return {red: 255, green: 255, blue: 255}
case 18 > $c return {red: 0, green: 0, blue: 255}
case 23 > $c return {red: 255, green: 255, blue: 0}
case 30 > $c return {red: 255, green: 140, blue: 0}
default return {red: 255, green: 0, blue: 0}
 ) , "image/png" )


## Connections: dbc Class

We provide a "dbc" class to manage connections to rasdaman servers, and managing the sending and receiving of the data. User can create a dbc object just by calling the constructor with the server url.

We import dbc class of the wdc package

In [11]:
from wdc import dbc

# Create Connection object
connection = dbc("https://ows.rasdaman.org/rasdaman/ows")

print(connection)

### Sending Queries to rasdaman Servers

We can send a query by utilizing a Connection object's execute_query() method. This method accepts a query as its argument, and sends this query to rasdaman servers, returning the content of the response.

In [12]:
# Let us create a query from scratch
query = Query(["AvgLandTemp"])
query.set_return_value(42) # Query will return 42
response = connection.execute_query(query)

print(response)

42


## Managing Data with Datacubes: dco Class

The last class we will talk about is the "dco" class. This class manages the connections and queries together. It contains its own "dbc" object.

We can import it from the wdc package.

In [13]:
from wdc import dco

### Creating a Datacube Object

We can create a Datacube object by passing query and connection to the constructor.

In [14]:
# Creating a query for the coverage "AvgLandTemp"
# You can try to write down the equivalent WCPS query as an exercise
connection = dbc("https://ows.rasdaman.org/rasdaman/ows?&SERVICE=WCS&ACCEPTVERSIONS=2.1.0&REQUEST=GetCapabilities")
query = Query(["AvgLandTemp"])
datacube = dco(connection, query)

print(datacube)

### Process a Query of a Datacube

Users can process the query they provided to the "dcp" object they created. We use the execute_query() method of dco object, which will send the query to rasdaman, get back the result and return it.

This returns the data as bytes. This is useful for cases such as image data, however if a scalar value was returned by rasdaman, user needs to decode this data.

We can print the received data via the print() function.

If the return type of the data is an image (e.g JPEG or PNG), we can utilize the display_result() method of DataVisualizer class to display the data as an image. Note that this will raise an error if the data is a scalar or the server returns back an error message.

In our case, we set the query so that it asks for image. Therefore we will use the DataVisualizer.

In [15]:
datacube.query.set_subset([AxisSubset("ansi", "2014-07"),
                AxisSubset("Lat", -20, 30),
                AxisSubset("Long", -20, 30)])
datacube.query.encode(Query.Types.png)
response = datacube.connector.execute_query(query) # Can also do datacube.execute_query(query)

# Display the result using the DataVisualizer class
from wdc import DataVisualizer
DataVisualizer().display_result(response)

## Final Words

This concludes the quick tutorial to using this library. With this library, we can create from simple queries to complex one with coverage constructors and switch cases.

With its current capabilities, it allows user to generate WCPS queries with Python and users can manage most of their coverage tasks with this library. There is, of course, more room to improvement, such as implementing user-defined aggregation methods via "condense" keyword.